In [21]:
from kmodes.kprototypes import KPrototypes
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import itertools
from sklearn.ensemble import IsolationForest


In [22]:
path = '/Users/mehmetdeniz/Library/CloudStorage/GoogleDrive-mehmetbakideniz@gmail.com/My Drive/meta_audience_insights_age_data.csv'
df1=pd.read_csv(path)

In [42]:
def combined_set_anomaly(df, columns_category, columns_metric): 
# Define the columns for grouping and aggregation

    aggregations = {'cost': 'sum', 'clicks': 'sum', 'purchase_value': 'sum', 'install': 'sum', 'impressions': 'sum','purchase': 'sum' }
    
    # Initialize an empty list to store the grouped DataFrames
    grouped_dfs = []
    
    # Perform grouping operations for different combinations of columns
    for i in range(1, len(columns_category) + 1):
        combinations = itertools.combinations(columns_category, i)
        for combo in combinations:
            grouped_df = df.groupby(list(combo)).agg(aggregations).reset_index()
            
            grouped_dfs.append(grouped_df)
            
    df2 = pd.concat(grouped_dfs, axis=0, join='outer')
    df2['cpm'] = np.where(df2['impressions'] != 0, (df2['cost'] / df2['impressions'])*1000, 0)
    df2['cr'] = np.where(df2['clicks'] != 0, df2['purchase'] / df2['clicks'], 0)
    df2['cpi'] = np.where(df2['install'] != 0, df2['cost'] / df2['install'], 0)
    df2['ctr'] = np.where(df2['clicks'] != 0, df2['impressions'] / df2['clicks'], 0)
    df2['cpc'] = np.where(df2['clicks'] != 0, df2['cost'] / df2['clicks'], 0)
    df2['roas']=np.where(df2['cost'] != 0, df2['purchase_value'] / df2['cost'], 0)
    df2.drop(['clicks', 'purchase_value', 'cost', 'install', 'impressions','purchase'], axis=1, inplace= True )
    df2.reset_index(inplace=True, drop=True)
    df2['combinations'] = df2[columns_category].apply(lambda x: ', '.join(x.dropna().astype(str)), axis=1)
    df2 = df2.drop(columns=columns_category)
    #df2.drop(['index'], axis=1, inplace= True )
    
    
    # Create a new DataFrame with only the columns of interest
    df_subset = df2[columns_metric]
    
    # Create an instance of the IsolationForest model
    isolation_forest = IsolationForest(n_estimators=100,max_samples='auto',contamination=float(0.1))
    
    # Fit the model on the data
    isolation_forest.fit(df_subset)
    
    # Predict anomalies for the data points
    predictions = isolation_forest.predict(df_subset)
    
    # Add the "IsAnomaly" column to df2
    df2['IsAnomaly'] = predictions
    df2=df2[df2['IsAnomaly'] ==-1]
    df2 = df2.drop(columns='IsAnomaly')

    return df2


In [43]:
columns = ['gender', 'age', 'campaign_type', 'optimization_strategy']
columns_of_interest = ['ctr', 'cpi']
df_combinations=combined_set_anomaly(df1, columns, columns_of_interest )

In [47]:
df_combinations=df_combinations[columns_of_interest+['combinations']]

In [48]:
df_combinations

,ctr,cpi,combinations
27,74.601112,20.517435,"unknown, 18-24"
52,103.647800,195.152049,"18-24, LINK_CLICKS"
69,17.815863,593.034735,"55-64, OUTCOME_SALES"
74,103.647800,195.152049,"18-24, NONE"
91,100.509665,300.473088,"female, 18-24, LINK_CLICKS"
95,66.077894,554.171911,"female, 25-34, LINK_CLICKS"
104,12.968593,709.760115,"female, 45-54, OUTCOME_SALES"
108,13.770164,1015.363687,"female, 55-64, OUTCOME_SALES"
114,106.382980,152.068078,"male, 18-24, LINK_CLICKS"
137,146.083621,106.226122,"unknown, 18-24, LINK_CLICKS"
